# Data Pipeline Prototype

## Importing libraries

In [ ]:
# ! pip install pandas praw prawcore python-dotenv pyarrow

In [ ]:
import pandas as pd
import praw, prawcore, time, os, sys, functools, random
import datetime as dt
import pyarrow as pa
import pyarrow.parquet as pq
from dotenv import load_dotenv
from typing import List, Dict, Any, Union, Tuple
from itertools import product
from collections import deque, namedtuple
from collections.abc import Generator, Callable

## Setting up access to Reddit API
Access keys to Reddit API are stored in a .env file under the config directory of this repository. A template for the .env file is provided in the config directory.

The config.py script assigns the environment variables to the `PRAW_ID`, `PRAW_SECRET`, `PRAW_USER_AGENT`, `PRAW_USERNAME`, and `PRAW_PASSWORD` global variables respectively.  

In [ ]:
# Load .env file for access keys
load_dotenv(os.path.join('..', 'config', '.env'))

# Import config.py to access environment variables
sys.path.append('../config')
from config import PRAW_ID, PRAW_SECRET, PRAW_USER_AGENT, PRAW_USERNAME, PRAW_PASSWORD

In [ ]:
# Initialize PRAW 
reddit = praw.Reddit(
    client_id=PRAW_ID,
    client_secret=PRAW_SECRET,
    username=PRAW_USERNAME,
    password=PRAW_PASSWORD,
    user_agent=PRAW_USER_AGENT
)

## Extracting text data

### Defining Utility Functions

#### Decorator Factory for implementing Exponential Backoff and Full Jitter
`backoff_on_rate_limit`: This is a decorator factory that builds a custom decorator based on specified backoff parameters (max retries, base delay, cap, jitter). The decorator itself is a wrapper for custom functions that call PRAW methods such as `fetch_submissions` and `fetch_comments`, which call subreddit.search() and submission.comments.replace_more() respectively. The decorator implements exponential backoff with optional full jitter to respect Reddit API rate limits while handling transient failures.
<blockquote>

__Inputs:__
- Integer value for max retries. When attempts exceed this number, an Exception is raised
- Float for base delay in seconds (i.e. Delay at first failed attempt)
- Float for maximum delay in seconds
- Bool on whether to implement full jitter or not

__Outputs:__
- Decorator to be applied to an PRAW API request wrapper
</blockquote>

In [ ]:
def backoff_on_rate_limit(max_retries:int=5, 
                        base_delay:float=1.0, 
                        cap_delay:float=60.0, 
                        jitter:bool=True):
    """
    Decorator factory that applies exponential backoff (with optional jitter)
    when Reddit API rate limits (HTTP 429) or server errors occur.
    Stops after max_retries and re-raises the exception.
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # Start with base delay, then exponentially scale by attempt
            attempt = 0
            while True:
                try:
                    return func(*args, **kwargs)
                except prawcore.exceptions.ResponseException as e:
                    if attempt > max_retries:
                        raise Exception("Max retries exceeded with Reddit API.")
                    delay = min(cap_delay, base_delay * 2 ** attempt)
                    if jitter:
                        delay = random.uniform(0, delay)
                    print(f"[WARNING] {e.__class__.__name__} on attempt {attempt+1}, retrying after {delay:.2f}s.")
                    time.sleep(delay)
                    attempt += 1
        return wrapper
    return decorator

#### API Call Wrappers with Exponential Backoff and Full Jitter

The helper functions were designed to extract relevant data and metadata from Reddit submissions and comments, and package the data into a dict of dicts that can be easily parsed into a Pandas DataFrame object for further analysis. The backoff decorator is applied to each API call wrapper to handle transient errors raised by HTTP 429 response (Too Many Requests).

In [ ]:
@backoff_on_rate_limit()
def fetch_submissions(subreddit:object, query:str, limit:int=100, **kwargs):
    """Modify the subreddit search from PRAw to ensure adherence to safe request limits."""
    return subreddit.search(**kwargs, query=query, limit=limit)

@backoff_on_rate_limit()
def fetch_comments(submission:object, limit:int=0):
    """Modify the comment fetch from PRAW to ensure adherence to safe request limits."""
    # Replace 'more' with specified limit (default = 0 or retain top-level comments only)
    submission.comments.replace_more(limit=limit)
    for comment in submission.comments:
        yield comment

#### Rate Limiter Class

The RateLimiter class is initialized at the beginning of the script and is used to track API requests made within a specific sliding window. Requests are throttled when total expected requests go beyond rate limits (Reddit = 100/min) for the current window. Jitter is injected to the wait time  

In [ ]:
class RateLimiter:
    """Rate Limiter with sliding window implementation."""
    def __init__(self, max_requests:int=100, period:float=60.0, jitter:Union[List[float],Tuple]=(0.1,5.0)):
        self.max_requests = max_requests
        self.period = period
        self.jitter = jitter
        self.trace_requests = deque()
        
    def wait_for_slot(self, n_request:int=1) -> None:
        """
        Delays execution of subsequent API request or code chunk to ensure maximum
        function calls or request adheres to rate limits within a specified window.
        """
        window_end = time.time()
        window_start = window_end - self.period
        
        # Remove older batches when timestamp is out of current window
        while self.trace_requests and self.trace_requests[0] < window_start:
            self.trace_requests.popleft()
        
        # Check if additional request can be accommodated given requests made in current window
        if len(self.trace_requests) + n_request > self.max_requests:
            # Wait time is adjusted by jitter
            wait_time = (self.trace_requests[0] + self.period) - window_end + random.uniform(*self.jitter)
            time.sleep(max(wait_time, 0))
            # Re-run the function and determine if request can be accommodated
            return self.wait_for_slot(n_request)
        
        # Enqueue current request to trace requests
        for _ in range(n_request):
            self.trace_requests.append(time.time())

#### Data Streaming Functions

`stream_comments`: This is a utility function that fetches comments from a given post and formats each comment as a dictionary of dictionaries with key as comment id and value as a dictionary of comment content and metadata (e.g. body, timestamp, upvotes).
<blockquote>

__Inputs:__ 
- Submission object from PRAW (i.e. Reddit posts)
- Integer for .replace_more limit parameter, default=0 (i.e. top/parent comments only)

__Output:__
- Dict of comments in the format {comment_id : {data_header: data_value}}
</blockquote>

`stream_submissions_and_comments`: This is a utility function that fetches submissions (posts) from a given subreddit using a predefined search query (i.e. keywords). Submissions are formatted into a dict of dicts with format {submission id : {data_header : data_value}}. This returns a tuple of submission data and comment data.
<blockquote>

__Inputs:__ 
- String of Subreddit name
- String of search query
- Integer for limit of submissions yielded by PRAW subreddit search

__Output:__
- Tuple of submission data dict and comment data dict
</blockquote>

`stream_aggregate_results`: This function is a wrapper for the `stream_submissions_and_comments` generator function and takes a list of subreddit names and search queries to feed the subreddit-query pairs to the wrapped function. A time delay is included between every inner function call to ensure adherence to the 100 requests/minute rate limit.
<blockquote>

__Inputs:__ 
- List of subreddit name strings
- List of search query strings
- Int of maximum requests per minute, also determines upper bound of search result limit
- Int of minimum requests, which is the floor of search result limit
- Float of seconds denoting the time period for counting the API call limits
- List of float values of seconds to randomly add to interval delay 

__Output:__
- Tuple of aggregated submissions dict and comments dict
</blockquote>

__Read more:__
1. [Rate Limiter - Sliding Window Counter](https://medium.com/@avocadi/rate-limiter-sliding-window-counter-7ec08dbe21d6)

__Read more:__
1. [API Rate Limits Explained: Best Practices for 2025](https://orq.ai/blog/api-rate-limit)
2. [Exponential Backoff And Jitter](https://aws.amazon.com/blogs/architecture/exponential-backoff-and-jitter/)
3. [Yield Statements vs. Returning Lists in Python](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://community.aws/content/2h01Byx1ytU8357tp2bvcUuJ2j0/yield-statements-vs-returning-lists-in-python%23:~:text%3DYield%253A%2520Ideal%2520for%2520large%2520data,potentially%2520leading%2520to%2520memory%2520errors.&ved=2ahUKEwjzvJvd74uOAxVkQ6QEHVAVMHcQFnoECBIQAw&usg=AOvVaw3hMoJHnPwBIQOdBmB_NiBD)

In [ ]:
def stream_comments(submission:object, limit:int=0):
    """
    Fetches comments from a Submission objects then parses each comment into a dictionary record.
    Each entry is streamed for efficient memory footprint when handling larger CommentForests.
    """
    # Dict of dicts with format {comment_id : comment_info_dict}
    comment_data: Dict[str, Dict[str, Any]] = {}
    # Update comments dict with info dict 
    for comment in fetch_comments(submission, limit=limit):
        record = {
            'comment_id':comment.id,
            'body':comment.body,
            'score':comment.score,
            'timestamp':comment.created_utc,
            'subreddit':comment.subreddit_name_prefixed,
            'parent_submission_id':submission.id
        }
        # Record API requests and stream comment data when slot available in current window
        rate_limiter.wait_for_slot()
        yield "comment", record

In [ ]:
def stream_submissions_and_comments(subreddit_name:str, query:str, limit:int=50, **search_kwargs):
    """
    Fetches submissions, parses each submission into a dictionary record, and calls the stream_comments
    function on each submission. Submission data and comment data are streamed for efficient memory 
    footprint when handling larger datasets. 
    """
    sub = reddit.subreddit(subreddit_name)
    
    # Fetch submissions, and for every submission, fetch the comments
    for submission in fetch_submissions(**search_kwargs, subreddit=sub, query=query, limit=limit):
        # Update submissions dict with info dict from submission
        record = {
            'submission_id':submission.id,
            'title':submission.title,
            'selftext':submission.selftext,
            'score':submission.score,
            'upvote_ratio':submission.upvote_ratio,
            'timestamp':submission.created_utc,
            'subreddit':submission.subreddit_name_prefixed,
            'num_comments':submission.num_comments
            }
        # Stream comment data from current submission ("submission", Dict[str, Any])
        yield from stream_comments(submission)
        # Record search request and stream submission data when slot available in current window
        rate_limiter.wait_for_slot()
        yield "submission", record

In [ ]:
def stream_aggregate_results(subreddits:List[str], 
                             queries:List[str],
                             max_requests:int=100, 
                             min_requests:int=50,
                             period:float=60.0,
                             jitter:List[float] = [1.0,10.0],
                             verbose:bool=False,
                             **search_kwargs):
    """
    Wrapper for streaming functions. Takes a list of subreddits and queries, then calls the 
    stream_search_results  function for each combination of subreddit and query. Jitter is implemented 
    to introduce randomness in number of API requests with a short backoff in each iteration to ensure
    adherence to Reddit API rate limits.
    """
    assert isinstance(subreddits, list), "Argument 'subreddits' expects a list of subreddit names."
    assert isinstance(queries, list), "Argument 'queries' expects a list of search queries names."
    
    # API request counter for triggering execution cooldown
    Trace = namedtuple('Trace', ['timestamp','total_requests'])
    trace_requests = []
    total_requests = 0
    
    # Parse submission and comment data with jittered API calls
    for subreddit, query in product(subreddits, queries):
        # Random number of requests per iteration to reduce predictability
        submission_limit = int(random.uniform(min_requests, max_requests))
        
        # Stream data 
        yield from stream_submissions_and_comments(**search_kwargs, subreddit_name=subreddit, query=query, limit=submission_limit)
        
        # Update total requests and request history
        total_requests += submission_limit
        trace_requests.append(Trace(f'{dt.datetime.now():%H:%M:%S}', total_requests))
    
    if verbose:
        print(f'Finished writing data to disk. Total requests made: {sum(trace_requests)}')
        print(f'-------Request Trace-------\n{"\n".join(trace_requests)}.')
        

#### Text File Parser for Subreddit and Search Queries

`parse_txt_file`: Parses text files containing data separated by newlines. Returns a list. Used for containerizing search_queries and subreddit strings into separate text files that can be easily mutated without modifying source code.
<blockquote>

__Input:__
- String for the path of text file, with each item separated by a newline

__Output:__
- List (e.g. search queries, subreddit names)
</blockquote>

In [ ]:
def parse_txt_file(file_path:str):
    """
    Utility function for parsing a multi-line text file where each item is separated
    by a newline.
    Input: String for file path
    Output: List
    """
    with open(file_path, 'r') as f:
        # Ignore comments and empty lines
        results = [line.rstrip("\n") for line in f if not (line.startswith('#') or line.startswith("\n"))]
    return results

#### Data Storage Function

Data will be stored as Pyarrow tables since Parquet files have higher compression rates resulting in smaller memory footprint, which is extremely beneficial for large datasets.

In [273]:
# Defining schemas for storing submission and comment data
SUBMISSION_SCHEMA = pa.schema([
    ("submission_id", pa.string()),
    ("title", pa.string()),
    ("selftext", pa.string()),
    ("score", pa.int32()),
    ("upvote_ratio", pa.float16()),
    ("timestamp", pa.timestamp("s")),
    ("subreddit", pa.string()),
    ("num_comments", pa.string()),
])

COMMENT_SCHEMA = pa.schema([
    ("comment_id", pa.string()),
    ("body", pa.string()),
    ("score", pa.int32()),
    ("timestamp", pa.timestamp("s")),
    ("subreddit", pa.string()),
    ("parent_submission_id", pa.string()),
])

# Defining paths for parquet files
PARQUET_PATHS = {
    'submission' : os.path.join('..','data','submission_data.parquet'),
    'comment' : os.path.join('..','data','comment_data.parquet')
    }

In [ ]:
def write_to_parquet(stream: Callable, file_paths:Dict[str], target_mb:float=8.0, *stream_args, **stream_kwargs):
    """TO FINISH"""
    submission_buffer: Dict[str, list] = {name:[] for name in SUBMISSION_SCHEMA.names}
    comment_buffer: Dict[str, list] = {name:[] for name in COMMENT_SCHEMA.names}
    buffer_map = {'submission':submission_buffer,'comment':comment_buffer}
    schema_map = {'submission':SUBMISSION_SCHEMA,'comment':COMMENT_SCHEMA}
    submission_writer = pq.ParquetWriter(file_paths['submission'], schema=SUBMISSION_SCHEMA)
    comment_writer = pq.ParquetWriter(file_paths['comment'], schema=COMMENT_SCHEMA)
    
    for i, (record_type, record) in enumerate(stream(*args, **kwargs)):
        # Determine pre-compiled writer
        buffer = buffer_map[record_type]
        if record_type == "submission":
            writer = submission_writer
        elif record_type == "comment":
            writer = comment_writer
        # Map fields from records to specific buffer
        for col in schema_map[record_type].names:
            buffer[col].append(record.get(col))
        # When buffer size reaches target byte size, write to disk
        if i % 1e3 == 0 and len(json.dumps(buffer)) // 1e6 >= target_mb:
            pa.RecordBatch.from_pydict(buffer)

In [326]:
pa_recordbatch = pa.RecordBatch.from_pydict(buf)

In [314]:
buf = {col:[] for col in submission_data_pqt.columns}
buf.update({'submission_id':[]})
for k,v in abc.items():
    buf['submission_id'].append(k)
    for key in buf.keys():
        if key != 'submission_id':
            buf[key].append(v[key])  

In [322]:
%%timeit
len(json.dumps(buf))

556 μs ± 11.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [250]:
import json

In [291]:
json.dumps(abc)

'{"1d0xqjx": {"title": "Most reliable used car, under $25k, less than 10 years old?", "selftext": "I\\u2019m thinking it\\u2019s likely a Toyota or Honda, but curious if you were looking to buy, what would you be looking for that meets the criteria? ", "score": 29, "upvote_ratio": 0.94, "timestamp": 1716716001.0, "subreddit": "r/CarsAustralia", "num_comments": 120}, "1jx4l2a": {"title": "Used SUV under $20k (need a reliable car ASAP)", "selftext": "_Reposting this because I didn\\u2019t get any comments on my previous post :( I really need to get a new car soon because my current one is stressing me out so please help a gal out \\ud83d\\ude2d\\ud83d\\ude4f\\ud83c\\udffc_\\n\\nHey yall, I\\u2019m a P-plater in my 20s based in Brisbane, currently looking to replace my car. I\\u2019m after a pre-owned automatic small/compact SUV that is around $20k or less, under 100,000 km, around 5 years old or newer, good fuel economy (for long-distance and highway driving), and higher ground clearance

In [290]:
%%timeit
len(json.dumps(sub_data_json))

437 μs ± 7.96 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [289]:
%%timeit
sys.getsizeof(json.dumps(abc))

628 μs ± 7.49 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [284]:
%%timeit
sys.getsizeof(sub_data_json)

63.7 ns ± 0.936 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [242]:
submission_data_pqt = pd.read_parquet("../data/submission_data_0.parquet")

In [270]:
abc = submission_data_pqt.to_dict(orient='index')

In [275]:
records = [v for k,v in abc.items()]

In [277]:
titles = [record['title'] for record in records]

### Fetching and parsing search results from Reddit used car communities

To scrape the relevant text data from Reddit, I created a small list of queries covering diverse yet relevant topics to buying affordable used vehicles. The queries involved location-specific, model-specific, and thematic keywords to ensure that the search covers as much ground as possible. Chosen subreddits have > 1e5 subscribers to ensure that search queries will yield a significant amount of results per API request.

With a 10x10 query and subreddit array, I expect at least an initial 100 requests for the subreddit search yielding 100x100 submissions at most.

Fetching the comments involves significantly more requests as each submission requires 1 request to yield the CommentForest. Fetching the comments will require at least 10,000 requests.

__Expected Minimum API Requests__
|Search Requests|Comment Fetch Requests|Total Requests|
|:----------|:----------|:----------|
|100      |10,000  |10,100|

As such, a single batch job covering all query-subreddit combinations will yield at least 10,100 API requests in a single go, which wildly exceeds the Reddit API fair use policy (i.e. Cap requests to 100/min averaged over 10-minute sliding window). To address this issue, batched processing will be implemented to ensure average requests is under safe rate limits.

In [338]:
# Parse text files containing search queries and subreddit names
search_queries = parse_txt_file("../src/search_queries.txt")
subreddits = parse_txt_file("../src/subreddits.txt")

In [ ]:
search_queries = search_queries[0]
subreddits = subreddits[0]
print('-------Search Pairs-------')
for (subreddit, query) in product(subreddits, search_queries):
    print(subreddit,"-",query)

-------Search Pairs-------
CarsAustralia - affordable reliable used cars under 15k Australia
CarsAustralia - affordable reliable used cars under 10k USA
UsedCars - affordable reliable used cars under 15k Australia
UsedCars - affordable reliable used cars under 10k USA


In [344]:
%%time
# Fetch search results and parse to dict of dicts
submission_data, comment_data = aggregate_search_results(subreddits=subreddits, queries=search_queries)

Cooldown triggered: sleeping for 109.40741414375071s to avoid rate limit.
Cooldown triggered: sleeping for 71.57586797057071s to avoid rate limit.
Cooldown triggered: sleeping for 65.00197525949584s to avoid rate limit.
CPU times: user 2.8 s, sys: 314 ms, total: 3.11 s
Wall time: 7min 44s


## Storing the scraped data

### Formatting to a Pandas DataFrame

In [348]:
submission_df = pd.DataFrame.from_dict(submission_data, orient='index')
comment_df = pd.DataFrame.from_dict(comment_data, orient='index')

### Exporting DataFrame to a Parquet file for efficient storage

In [349]:
submission_df.to_parquet(os.path.join("..","data","submission_data.parquet"), 
                         engine='pyarrow',
                         compression='gzip')

comment_df.to_parquet(os.path.join("..","data","comment_data.parquet"),
                      engine='pyarrow',
                      compression='gzip')

## Exploratory Data Analysis

## Test

In [ ]:
rate_limiter = RateLimiter(max_calls=3, period=2, jitter=[1,2])

def bar():
    yield from foo()

def foo():
    for j in [i for i in range(15)]:
        rate_limiter.wait_for_slot(n_request=1)
        yield j
        
for k in bar():
    print(k)